<a href="https://colab.research.google.com/github/yogso/pbil_hiperarameters_tunning/blob/master/Hiperparameters%20Tunning%20via%20PBIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stldecompose

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import random as rn
import math

from keras.backend import clear_session
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, concatenate, Input
from keras.layers import Convolution1D, MaxPooling1D
from keras.optimizers import Adam

from sklearn.metrics import mean_squared_error

import matplotlib as mpl
mpl.rcParams['xtick.labelsize'] = 'large'
mpl.rcParams['ytick.labelsize'] = 'large'
mpl.rcParams['axes.labelsize'] = 'large'

"""The imports: personal libraries"""
#np.set_printoptions(threshold=np.inf)

from stldecompose import decompose, forecast   #pip install stldecompose
from stldecompose.forecast_funcs import (naive,
                                         drift, 
                                         mean, 
                                         seasonal_naive)

Using TensorFlow backend.


In [0]:
dataset = [90, 79, 99, 117, 99, 99, 86, 95, 93, 69, 87, 94, 74, 90, 76, 71, 87, 60, 72, 73, 77, 51, 66, 58, 63, 52.5, 67, 63, 78, 84, 69, 75, 
77, 71, 82, 85, 82, 81.5, 94, 99, 97, 78, 93.5, 80, 92, 74, 71, 83, 70, 80, 86, 61, 77.6969111969112, 70, 83, 95, 82, 86, 83, 83, 82, 79, 101, 
122, 100, 74, 70, 70, 70, 74, 73, 83, 66, 60, 66, 62, 60, 69, 67, 71, 68, 60, 68, 73, 66, 67, 72, 77, 67, 47, 68, 85.5, 84, 78, 89, 81, 61, 
75, 99, 104, 83, 77.6969111969112, 62, 70, 91, 98, 103, 112, 105, 111, 109, 99, 110, 88, 82.5, 99, 81, 79, 72, 80, 75, 86, 77, 61, 56, 55, 66, 
60, 71, 71, 74, 72, 54, 65, 74, 75, 76, 72, 69, 78.5, 67, 72, 63, 69, 87, 71, 71, 72.5, 75, 93, 89, 100, 96, 96, 101, 102, 70.5, 72, 74, 67, 
68, 70, 65, 75.5, 72, 65, 80, 95, 94.5, 71, 84.5, 81, 78, 71, 66, 83, 85, 62, 73, 80, 69, 66, 63, 63, 69, 68, 78.5, 78, 78, 79, 67, 69, 82, 
78, 61, 73.5, 70, 79, 81.5, 83, 90, 79, 99, 97, 95, 67, 79.5, 65, 80, 74, 70.5, 79, 78, 104, 77, 74, 87, 84, 94, 109, 91, 93.5, 95, 76, 72, 
61, 57, 59, 70, 68, 82, 67, 69, 73, 76, 70, 57, 75, 63, 72, 64, 66, 70, 81, 68, 74, 72, 79, 84, 81, 69, 77, 74, 97, 103, 107, 88, 96, 101]

date = pd.date_range(start='7/1/2013', freq='W', periods=259)
df = pd.DataFrame(dataset, index=date)

In [4]:
len(dataset)

259

In [0]:
# Hiperparametros a optimizar [Nombre del hiperparametro, rango desde <=, rango hasta >=, con paso]


hiperparametros = [
    ["batch_size", 40, 71, 1],
    ["lo_frac", 0, 1, 0.1],
    ["period", 30, 61, 1],
    ["lo_delta", 0, 0.2, 0.01],
    
    #Sample_size debe ser minimo 52, debido a que como en cada capa de la red convolucional se reduce la dimensionalidad,
    #entonces 52 es la minima dimension para que existan elementos en la capa de salida. Por este mismo motivo los kernels deben ser pequeños
    ["sample_size", 52, 100, 2],
    ["epoch", 20, 140, 20],
    ["learning_rate", 0.001, 0.05, 0.005],
    ["lr_decay", 0.0, 0.001, 0.0001],
    ["kernel1", 1, 10, 1],
    ["kernel2", 1, 6, 1],
    ["kernel3", 1, 3, 1],
    ["filtro1", 10, 50, 5],
    ["filtro2", 30, 80, 10],
    ["filtro3", 60, 160, 20]
]



##Valores por defecto:
#lo_delta=0.01
#batch_size = 50
#period = 5
#lo_frac = 0.6
#sample_size = 52
#epoch=100
#learning_rate=0.001
#lr_decay=0.0
#kernel1 = 9
#kernel2 = 5
#kernel3 = 3
#filtro1 = 32
#filtro2 = 64
#filtro3 = 128



In [0]:
import gc

generaciones = pd.DataFrame(columns=['Generacion','MSE'])

#generaciones = pd.read_pickle('data_mayo/generationes_periodo_batchsize_nuevo_3.pkl')

def pbil_optimizacion(learn_rate, learn_rate_neg, num_pob, num_mejores_para_actualizar_vec, num_peores_para_actualizar_vec, vec_len,
             ciclos_optimizacion, eval_f, eps=0.01, lista_vecs=None):
    """
    learn_rate: tasa de actualización del vector poblacional (vec) hacia cada uno de los mejores individuos.
    learn_rate_neg: similar a learn_rate, pero para alejar el vector de los peores individuos.
    num_pob: número de individuos en cada población
    num_mejores_para_actualizar_vec: ¿cuántos mejores individuos se utilizarán para actualizar el vector poblacional?
    num_peores_para_actualizar_vec: ¿cuántos de los peores individuos se utilizarán para actualizar el vector poblacional?
    vec_len: longitud del vector de población
    ciclos_optimizacion: numero de ciclos de optimizacion
    eval_f: función para la evaluación del fitness de cada individuo
    eps: el vector poblacional será alejado de los valores extremos (0, 1) sumando o restando un valor eps determinado.
    lista_vecs: almacena los vectores de población de cada generación.
    """
    global generaciones
    secciones = [row[2] for row in hiperparametros_int_aux]
    
    # Inicialización del vector
    vec = crea_prob_inicial(vec_len)

    # Inicialización de la población
    poblacion = pob_ini(num_pob, vec_len)
    scores = scores_ini(num_pob)
    modelos = [None] * num_pob
    hipers = [None] * num_pob

    # Inicializar mejor resultado
    # Contendrá: [Valoración, vector binario del individuo, modelo entrenado, hiperparametros en decimales]
    mejor_de_todos = [float("inf"), None]

    lista_vecs = [vec]

    for i in range(ciclos_optimizacion):
        for j in range(num_pob):
      #      poblacion[j] = get_num(vec)

            poblacion[j][0:secciones[0]] = get_num(vec[0:secciones[0]])
            anterior = secciones[0]
            indx=1
            while indx < len(secciones):
                poblacion[j][anterior:secciones[indx]+anterior] = get_num(vec[anterior:secciones[indx]+anterior], indx= indx)
                anterior = anterior + secciones[indx]
                indx+=1
            
            # Evaluaciones de los individuos
            scores[j], modelos[j], hipers[j] = eval_f(poblacion[j])
        # Selección de los mejores indiviuos
        results_ordenados = sorted(zip(scores, poblacion, modelos, hipers), key=lambda x:x[0], reverse=False)
        mejor = results_ordenados[:num_mejores_para_actualizar_vec]
        worst = results_ordenados[-num_peores_para_actualizar_vec:]

        if mejor_de_todos[0] > mejor[0][0]:
            mejor_de_todos = (mejor[0][0], list(mejor[0][1]), mejor[0][2], mejor[0][3])
            
        print('Paso: {0}'.format(i))
        print('Vector de prob: {0}'.format(vec))
        print('Mejores: {0}'.format([(b[0]) for b in mejor]))
        print('Peores: {0}'.format([(w[0]) for w in worst]))
        print('Mejor de todos: {0}'.format((mejor_de_todos[0])))
        print("--------------")
        
        # Actualizar vector
        for v in mejor:
            vec += 2 * learn_rate * (v[1] - 0.5)
        for v in worst:
            vec -= 2 * learn_rate_neg * (v[1] - 0.5)

        # Corrección del vector si hay elementos fuera del rango [0, 1]
        for j in range(vec_len):
            if vec[j] < 0:
                vec[j] = 0 + eps
            elif vec[j] > 1:
                vec[j] = 1 - eps

        # Añadir vec
        lista_vecs.append(vec)
        
        generaciones = generaciones.append({'Generacion':i+1,
                            'MSE':mejor_de_todos[0]}, ignore_index=True)
        
        #generaciones.to_pickle('data_mayo/generationes_periodo_batchsize_nuevo_3.pkl')
                
    return mejor_de_todos

In [0]:
#Inicialización y entrenamiento de cada modelo

def model_creation(df, lo_delta=0.01, batch_size = 50, period = 5, lo_frac = 0.6, sample_size = 52, 
                   epoch=100, learning_rate=0.001, lr_decay=0.0, kernel1 = 9, kernel2 = 5, kernel3 = 3, 
                   filtro1 = 32, filtro2 = 64, filtro3 = 128):
  
    """
    df: DataFrame de pandas con los valores de la serie temporal.
    #Hiperparametros STL
    period: periodicidad más significativa de la serie temporal observada, en unidades de 1 observación. Ejm: para indicar una fuerte periodicidad anual con observaciones tomadas diariamente: period=365
    lo_frac: fracción de datos a utilizar en el entrenamiento de la regresión Lowess. 
    lo_delta: distancia fraccional dentro de la cual se puede utilizar la interpolación lineal en lugar de una regresión ponderada. Usar un lo_delta que no sea cero disminuye significativamente tiempo de cálculo.
    #Hiperparametros modelo
    batch_size: número de ejemplos de entrenamiento utilizados en cada iteración del entrenamiento.
    sample_size: número de observaciones previas que se tomarán como inputs.
    epoch: número de veces en que se procesarán todos los ejemplos del conjunto de datos en una sesión de entrenamiento.
    learning_rate: medida en la que los pesos se actualizan con respecto a la nueva información adquirida tras procesar cada batch.
    lr_decay: tasa de decaimiento del learning_rate en cada iteración
    kernel1-3: tamaño de la ventana (o detector de caracteristicas) que recorrerá los datos en cada capa de convolución.
    filtro1-3: número de filtros a utilizar en cada capa de convolución.
    """
  
    
    import numpy as np
    np.random.seed(42) # para reproducibilidad   
    import tensorflow as tf
    import keras
    #session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
    #                              inter_op_parallelism_threads=1)
    from keras import backend as K
    tf.set_random_seed(1234)
    #sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    #K.set_session(sess)    
    
    
    from tensorflow.python.util import deprecation
    deprecation._PRINT_DEPRECATION_WARNINGS = False
    
    
    ##DESCOMPOSICIÓN STL
    
    stl = decompose(df, period=period, lo_frac=lo_frac, lo_delta=lo_delta)  
    
    seasonal, trend, random =  np.asarray(stl.seasonal.values), np.asarray(stl.trend.values), np.asarray(stl.resid.values)
    
    #DEFINICIÓN DEL MODELO

#    sample_size = 52 # features (in this case, previous information you want consider for your predictions)

    #kernel_sz = [9,5,3]
    
    kernel_sz = [int(kernel1), int(kernel2), int(kernel3)]
    
#    filtros = [32,64,128]

    filtros = [filtro1,filtro2,filtro3]
    
    # CNN para seas
    inputseas = Input(shape=(sample_size, 1))
    convseas = Convolution1D(filtros[0], kernel_sz[0], activation='relu')(inputseas)
    convseas = MaxPooling1D(pool_size =2)(convseas)
    convseas = Convolution1D(filtros[1], kernel_sz[1], activation='relu')(convseas)
    convseas = MaxPooling1D(pool_size =2)(convseas)
    convseas = Convolution1D(filtros[2], kernel_sz[2], activation='relu')(convseas)
    convseas = MaxPooling1D(pool_size =2)(convseas)
    convseas = Flatten()(convseas)


    # CNN para trend
    inputtrend = Input(shape=(sample_size, 1))
    convtrend = Convolution1D(filtros[0], kernel_sz[0], activation='relu')(inputtrend)
    convtrend = MaxPooling1D(pool_size =2)(convtrend)
    convtrend = Convolution1D(filtros[1], kernel_sz[1], activation='relu')(convtrend)
    convtrend = MaxPooling1D(pool_size =2)(convtrend)
    convtrend = Convolution1D(filtros[2], kernel_sz[2], activation='relu')(convtrend)
    convtrend = MaxPooling1D(pool_size =2)(convtrend)
    convtrend = Flatten()(convtrend)

    # CNN para rand
    inputrand = Input(shape=(sample_size, 1))
    convrand = Convolution1D(filtros[0], kernel_sz[0], activation='relu')(inputrand)
    convrand = MaxPooling1D(pool_size =2)(convrand)
    convrand = Convolution1D(filtros[1], kernel_sz[1], activation='relu')(convrand)
    convrand = MaxPooling1D(pool_size =2)(convrand)
    convrand = Convolution1D(filtros[2], kernel_sz[2], activation='relu')(convrand)
    convrand = MaxPooling1D(pool_size =2)(convrand)
    convrand = Flatten()(convrand)

    merged = concatenate([convseas, convtrend, convrand],axis=1)
    out = Dense(64)(merged)
    out = Dense(16)(out)
    out = Dense(1, activation='linear')(out)

    final_model = Model(inputs=[inputseas, inputtrend, inputrand], outputs=[out])

    # Compilar
    adam = Adam(lr=learning_rate, decay=lr_decay)
    final_model.compile(loss="mse", optimizer=adam)

    ####
    
    # ENTRENAMIENTO DEL MODELO

    
    dataset = np.asarray(df.values)
    
    #seasonal = np.asarray(seasonal)
    #trend = np.asarray(trend)
    #random = np.asarray(random)

    y = dataset[sample_size:]

    Xseas = np.atleast_3d(np.array([seasonal[start:start + sample_size] for start in range(0, seasonal.shape[0]-sample_size)]))
    Xtre = np.atleast_3d(np.array([trend[start:start + sample_size] for start in range(0, trend.shape[0]-sample_size)]))
    Xrand = np.atleast_3d(np.array([random[start:start + sample_size] for start in range(0, random.shape[0]-sample_size)]))

    test_size = int(len(dataset)*0.3)

    trainY, testY = y[:-test_size], y[-test_size:]

    trainXseas = Xseas[:-test_size]
    testXseas =  Xseas[-test_size:]

    trainXtre = Xtre[:-test_size]
    testXtre =  Xtre[-test_size:]

    trainXrand = Xrand[:-test_size]
    testXrand =  Xrand[-test_size:]

    nextSteps = np.empty((1,sample_size,1))
    nextSteps[0,:,:]= np.atleast_3d(np.array([dataset[start:start + sample_size]
                                              for start in range(dataset.shape[0]-sample_size,dataset.shape[0]-sample_size+1)]))

    final_model.fit([trainXseas,trainXtre,trainXrand], trainY, epochs=epoch, batch_size=batch_size, verbose=0) 
    
    pred = final_model.predict([testXseas,testXtre,testXrand])

    testScore = mean_squared_error(testY, pred)
    
    clear_session()
    gc.collect()
    
    return testScore, final_model

In [0]:
import numpy as nump

##Devuelve el numero de bits minimos necesarios para representar un número decimal.

def num_bits(num_dec):    
    res= 2**(num_dec - 1).bit_length()
    if num_dec == res: 
        num_dec+=1
        res= 1 if num_dec == 0 else 2**(num_dec - 1).bit_length()
    res= int(res/2)
    
    return len(entero_a_binario(res))

#Toma un resultado obtenido en decimales y lo ubicar dentro del rango definido para su hiperparametro correspondiente.
def resultado_a_rango(num_dec, hiperparametro):
    resul = num_dec * hiperparametro[3] + hiperparametro[1]
    if resul>hiperparametro[2]: resul = hiperparametro[2]
    return resul


#Convierte una cadena de números binarios a código Gray. 
def binario_a_gray(num): 
    return num ^ (num >> 1) 

#Convierte una cadena de números binarios en formato Gray a entero.
def gray_a_entero(num_gray):
    num = binario_a_entero(num_gray)
    inv = 0; 
    while(num): 
        inv = inv ^ num; 
        num = num >> 1; 
    return inv;
    
def entero_a_binario(num):
    return [int(x) for x in bin(num)[2:]]
    
#Convierte una cadena binaria a una variable entera
def binario_a_entero(num_bit):
    out = 0
    for bit in num_bit:
        out = (out << 1) | bit
    return out

#Genera números aleatorios y los compara con los valores de un vector "prob"
def get_num(prob, indx=0):
    vec = [None] * len(prob)
    """
    Returns 0 (False) or 1 (True) depending on prob, if all 0s then run again
    """
    for i in range(len(prob)):
        vec[i]= rn.random() < prob[i]

    if indx ==1:
        if (gray_a_entero(vec)>10):
            vec = get_num(prob)
    return vec

#Creamos un array de longitud vec_len, correspondiente al vector de probabilidades.
#Al principio la probabiidad de 1 o 0 es 0.5 siempre.
def crea_prob_inicial(vec_len):
    return nump.full(vec_len, 0.5, dtype=float)

#Inicializa la población (con valores 0).
def pob_ini(num_pob, vec_len):
    return nump.zeros((num_pob, vec_len), dtype=int)

#Inicialización de los valores de la función fitness (comienzan valiendo None)
def scores_ini(num_pob):
    return [None for _ in range(num_pob)]

#Procesa el vector binario de un individuo y devuelve los valores de sus hiperparametros en decimales.

def proc_bin(bin_vector):
    global hiperparametros, hiperparametros_int_aux
    flag=0
    dec_resul=[]
    for i, hiper in enumerate(hiperparametros):
        dec_resul.append(resultado_a_rango(gray_a_entero(bin_vector[flag : flag + hiperparametros_int_aux[i][2]]),hiper))
        flag += hiperparametros_int_aux[i][2]
        
    return dec_resul

#Funcion encargada de procesar el vector binario de cada individuo, definir y entrenar su modelo correspondiente, y devolver su MSE. 

def fitness(bin_num):
  
    # dec_resul convierte el número binario de cada hiperparametro a una lista con su correspondencia en decimal
    dec_resul = proc_bin(bin_num)
    
    hiper_inputs = {}
    for i, v in enumerate(dec_resul):
        hiper_inputs[hiperparametros_int_aux[i][0]] = v
    
    loss, modelo = model_creation(df, **hiper_inputs)
   
    gc.collect()
    
    return loss, modelo, hiper_inputs


#Convertir la lista de hiperparametros a una lista con el nombre, el número total de posibles valores, y el número de
#bits minimo necesario para representarlo

hiperparametros_int_aux= []
for h in hiperparametros:
    hiperparametros_int_aux.append([h[0], math.ceil((h[2]-h[1])/h[3]), num_bits(math.ceil((h[2]-h[1])/h[3]))])

In [9]:
mejor = pbil_optimizacion(learn_rate=0.02,
                   learn_rate_neg=0.02,
                   num_pob=15,
                   num_mejores_para_actualizar_vec =3,
                   num_peores_para_actualizar_vec =3,
                   vec_len=sum([h[2] for h in hiperparametros_int_aux]),
                   ciclos_optimizacion=20,
                   eval_f=fitness)

W0706 00:17:12.131129 139847339415424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0706 00:17:12.132940 139847339415424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0706 00:17:12.141703 139847339415424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0706 00:17:12.163319 139847339415424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0706 00:17:12.380431 139847339415424 deprecation_wrapp

Paso: 0
Vector de prob: [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
Mejores: [113.38805392255414, 140.81116324784793, 146.9333894041778]
Peores: [4882.691697753113, 5180.032360112789, 5803.169978607052]
Mejor de todos: 113.38805392255414
--------------
Paso: 1
Vector de prob: [0.38 0.46 0.58 0.42 0.54 0.5  0.5  0.58 0.5  0.5  0.46 0.58 0.46 0.54
 0.5  0.5  0.5  0.54 0.54 0.46 0.5  0.46 0.5  0.46 0.54 0.5  0.5  0.5
 0.5  0.54 0.54 0.46 0.38 0.46 0.54 0.46 0.42 0.54 0.54 0.54 0.54 0.58
 0.46 0.58 0.54 0.5  0.5  0.42 0.42 0.46 0.5  0.5  0.54 0.46]
Mejores: [128.48149438615656, 148.08019737733977, 148.35242624889284]
Peores: [3035.6754279991046, 4075.558147659851, 5731.579538640197]
Mejor de todos: 113.38805392255414
--------------
Paso: 2
Vector de prob: [0.38 0.46 0.58 0.46 0.54 0.5  0.58 0.66 0.5  0.5  0.5  0.66 0